# Engine torque map

Makes it easy to plot the engine map.

In [ ]:
import math
import itertools
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

def rpm_to_rads(rpm):
    return rpm / 60 * (2.0 * math.pi)

def friction(coloumb_torque, viscous_coeff, rpm):
    return -(coloumb_torque + viscous_coeff \
             * rpm_to_rads(rpm))

rpms = [0., 400., 500., 1000., 1500., 2000., 2500., 3000., 3500., 4000., 4500., 5000., 5500., 6000.]
throttles = [0.0, 1.0]
coloumb_t = 50.0
visc_coeff = 0.06
brake_torque = [friction(coloumb_t, visc_coeff, rpm) for rpm in rpms]

max_eng_torque = 150.0
t = lambda x: max_eng_torque * x
max_torque = [friction(coloumb_t, visc_coeff, 0), friction(coloumb_t, visc_coeff, 400), t(0.0),
              t(0.4), t(0.6), t(0.8), t(0.9), t(0.95), t(1.0), t(0.99), t(0.93),
             t(0.85), t(0.75), t(0.6)]

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel("Engine angular velocity(RPM)")
ax.set_ylabel("Torque(Nm)")
ax.plot(rpms, brake_torque, label="Throttle 0%")
ax.plot(rpms, max_torque, label="Throttle 100%")
ax.plot(rpms, [rpm_to_rads(rpm) * torque / 1000 \
               for (rpm, torque) in itertools.zip_longest(rpms, max_torque)], label="Power(kW)")
ax.legend()

In [ ]:
# Ideal traction hyperbola
%matplotlib notebook
# Constants
wheel_radius = 0.344
min_rpm = 800
max_rpm = 5500
max_vel = 200.0
gears = [3.8, 2.6, 1.8, 1.29, 1.0, 0.8]
diff = 3.2


min_vel = (rpm_to_rads(min_rpm) * wheel_radius / (gears[0] * diff))
vel_range = np.linspace(min_vel, max_vel / 3.6, 100)
rpm_range = np.linspace(min_rpm, max_rpm, 100)
torque_range = np.interp(rpm_range, rpms, max_torque)

max_power = max([t * rpm_to_rads(v) for (t, v) in itertools.zip_longest(torque_range, rpm_range)])

def g(gear):
    vel = []
    force = []
    tot_gear = gear * diff
    
    for (c, x) in itertools.zip_longest(torque_range, rpm_range):
        force.append((c * tot_gear) / wheel_radius)
        vel.append(rpm_to_rads(x) * wheel_radius / tot_gear)  
    return (force, vel)

fig = plt.figure()
fig.suptitle("Traction hyperbola")
ax = fig.add_subplot()
ax.set_xlabel("Velocity (km/h)")
ax.set_ylabel("Force (kN)")
ax.set_ylim([0.0, max(g(gears[0])[0]) * 1e-3 + 1])
ax.set_xlim([min_vel, max_vel])
ax.plot(vel_range * 3.6, (max_power / vel_range) * 1e-3)

for (f, v) in [g(gear) for gear in gears]:
    ax.plot(np.array(v) * 3.6, np.array(f) * 1e-3)
    
print(f"Power = {max_power / 1000}kW")

for (i, gear) in enumerate(gears):
    print(f"{i+1} Min = {g(gear)[1][0] * 3.6:.4}km/h. Max = {g(gear)[1][-1] * 3.6:.4}km/h")

In [ ]:
# Theoretical cdA constant
air_density = 1.2041
# TODO: Exclude tire model rolling resistance
vel = vel_range[-1]
f = max_power / vel
print(f"cdA = {f / (0.5 * air_density * vel * vel)}")

In [ ]:
# C code
print(f"Table torque_map = table_with_capacity({len(throttles)}, {len(rpms)});")
for (i, throttle) in enumerate(throttles):
    print(f"torque_map.x[{i}] = {throttle};")

for (i, rpm) in enumerate(rpms):
    # angular velocity must be in rad/s
    print(f"torque_map.y[{i}] = {rpm_to_rads(rpm)};")

for (i, bt) in enumerate(brake_torque):
    print(f"torque_map.z[0][{i}] = {bt};")

for (i, mt) in enumerate(max_torque):
    print(f"torque_map.z[1][{i}] = {mt};")